In [1]:
import csv
from dataclasses import dataclass
from datetime import datetime
from typing import Optional

import pandas as pd
from IPython.display import clear_output
import pytz

/tmp/ipykernel_498921/571532899.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Open daa_snapshot.csv
# Contains daily daa/timestamps from genesis to Jan 24th 2024
# Used for estimating date via interpolation

@dataclass
class DAATimestamp:
    daa_score: int
    timestamp_ms: int
    datetime: datetime

daa_timestamps = []
with open('data/daa_timestamps.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # skip header row

    for row in reader:
        daa_timestamps.append(DAATimestamp(
            daa_score=int(row[0]), 
            timestamp_ms=int(row[1]), 
            datetime=row[2]
        ))

print(f'low snapshot daa: {daa_timestamps[0].daa_score} {daa_timestamps[-1].datetime}')
print(f'high snapshot daa: {daa_timestamps[-1].daa_score} {daa_timestamps[-1].datetime}')

low snapshot daa: 0 2024-01-24 08:59:58.71+00
high snapshot daa: 69694070 2024-01-24 08:59:58.71+00


In [3]:
def _interpolate_datetime(index, daa_score):
    current, next = daa_timestamps[index], daa_timestamps[index+1]
    score_difference = next.daa_score - current.daa_score
    
    if score_difference == 0:
        return None

    frac = (daa_score - current.daa_score) / score_difference
    interpolated_milliseconds = int(current.timestamp_ms + 
                                    (next.timestamp_ms - current.timestamp_ms) * frac)

    return pytz.utc.localize(datetime.fromtimestamp(interpolated_milliseconds / 1000))

In [4]:
def _binary_search_daa(daa_score):
    low, high = 0, len(daa_timestamps) - 1
    
    while low <= high:
        mid = (low + high) // 2
        mid_daa_score = daa_timestamps[mid].daa_score

        if mid_daa_score == daa_score:
            return mid

        if mid < len(daa_timestamps) - 1 and mid_daa_score <= daa_score < daa_timestamps[mid+1].daa_score:
            return mid

        if daa_score < mid_daa_score:
            high = mid - 1
        else:
            low = mid + 1

    return None

In [5]:
def estimate_datetime(daa_score):
    # TODO assuming your UTXO export contains DAAs higher than 
    # the most recent DAA in daa_timestamps.csv
    # Either get highest DAA from node or extrapolate
    
    # If daa_score = most recent daa_timestamp record, just return that datetime
    if daa_score == daa_timestamps[-1].daa_score:
        return daa_timestamps[-1].datetime

    index = _binary_search_daa(daa_score)
    if index is not None:
        return _interpolate_datetime(index, daa_score)

    return None

In [6]:
# Open, utxo_set.csv, iterate rows (utxos) and estimate date for each
# Csv fields are [address, daa_score, amount, is_coinbase]

# utxo_set.csv not included due to file size

# (lazily) storing results in memory

columns = ['daa_score', 'amount', 'estimated_datetime']
df_utxo = pd.DataFrame(columns=columns)
chunk = []
chunk_size = 10000 

with open('data/utxo_set.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header

    for i, row in enumerate(reader):
        if i % 1_000_000 == 0:
            clear_output()
            print(f'Processed {i:2,} UTXOs')
            
        if int(row[2]) <= 1000:  # Skip dust UTXOs
            continue

        utxo_daa = int(row[1])
        est_datetime = estimate_datetime(utxo_daa)

        # Append row data to the temporary chunk
        chunk.append({
            'daa_score': utxo_daa,
            'amount': int(row[2]),
            'estimated_datetime': est_datetime
        })

        # When the chunk reaches the chunk size, append it to the main DataFrame
        if len(chunk) >= chunk_size:
            df_utxo = pd.concat([df_utxo, pd.DataFrame(chunk)], ignore_index=True)
            chunk = []  # Clear the temporary list

print("Data loading complete.")

Processed 58,000,000 UTXOs
Data loading complete.


In [7]:
# failed to estimate timestamp for these records
# likely the result of utxo_daa being higher than the highest daa (daa_timestamps[-1].daa_score) in daa_timestamps.csv')
df_utxo[df_utxo['estimated_datetime'].isna()]

,daa_score,amount,estimated_datetime
220803,69694073,13859131548,NaT
2318401,63098762,4538409511,NaT
3249657,69694077,13859131548,NaT
3370102,69694077,13859131548,NaT
3391217,69694071,13859131548,NaT


In [8]:
# Get utxos created in 2022 
df_utxo['estimated_datetime'] = pd.to_datetime(df_utxo['estimated_datetime'])
df_2022_utxos = df_utxo[df_utxo['estimated_datetime'].dt.year == 2022]
df_2022_utxos[:10]

,daa_score,amount,estimated_datetime
306,33650095,8487634506,2022-12-03 20:30:30.192000+00:00
329,34929574,433103448,2022-12-18 15:43:47.790000+00:00
427,25062788,42999900508,2022-08-26 12:41:41.684000+00:00
466,27050116,39003694822,2022-09-18 11:55:14.162000+00:00
467,33647051,7544889254,2022-12-03 19:39:45.766000+00:00
469,27316340,200000000,2022-09-21 13:49:06.183000+00:00
553,27074118,447321356726,2022-09-18 18:34:10.323000+00:00
559,19764090,31646849,2022-06-26 08:03:07.105000+00:00
579,35680990,10004438088,2022-12-27 08:21:39.395000+00:00
580,35327182,10013109546,2022-12-23 06:07:55.991000+00:00


In [9]:
# Print every x step just to see what data looks like
df_utxo[::100_000]

,daa_score,amount,estimated_datetime
0,1490719,5000000000000,2021-11-27 20:20:54.490000+00:00
100000,65640007,900000,2023-12-08 22:03:20.825000+00:00
200000,60059149,12560824880,2023-10-05 08:42:16.325000+00:00
300000,28382065,10001664217,2022-10-03 21:48:03.729000+00:00
400000,60456785,804367845,2023-10-09 23:04:29.184000+00:00
500000,49172160,21802000000,2023-06-01 10:17:11.675000+00:00
600000,66009202,10004471086,2023-12-13 04:32:05.844000+00:00
700000,36001725,2079363491,2022-12-31 01:26:09.444000+00:00
800000,2328794,50000000000,2021-12-07 09:56:34.260000+00:00
900000,29313763,10097785684,2022-10-14 16:32:34.578000+00:00
